In [15]:
%load_ext autoreload
%autoreload 2
from motion import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
a = 1
b = 3
c = -2
d = 1
e = -2

def p(t):
  #return np.sin(4 * t)
  return a + b * t + c * t **2 + d * t**3 + e * t**4

In [17]:
num_ts = 10000
t_min = 0
t_max = 10
ts, psns, vels, accs = compute_psn_vel_acc(p, t_min, t_max, num_ts = num_ts)

In [18]:
make_figure(p, t_min, t_max)